<h3>Imports and Setup</h3>

In [11]:
import pandas as pd
import numpy as np

In [12]:
pd.set_option('display.max_columns', None)

In [104]:
big_table = pd.read_csv('data/T_T100_SEGMENT_ALL_CARRIER.csv')
area_codes_table = pd.read_csv('data/L_WORLD_AREA_CODES.csv')
carrier_classes = pd.read_csv('data/L_CARRIER_GROUP_NEW.csv')

In [105]:
# Replace World Area Codes (WAC) with descriptions from lookup table
big_table = big_table.merge(area_codes_table, left_on='DEST_WAC', right_on='Code', how='left')
big_table['DEST_WAC'] = big_table['Description']
big_table['ORIGIN_WAC'] = big_table['Description']
big_table = big_table.drop(columns=['Code', 'Description'])

In [59]:
big_table

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP_NEW,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,32250,HYA,"Hyannis, MA",Massachusetts,United States,Massachusetts,1,125,1,2024,1,1,1,F,DU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314502,891.0,889.0,2367000.0,8002.0,4689.0,5250.0,0.0,59.0,33895.0,26497.0,GV,Grant Aviation,D,GV,Grant Aviation,5,31555,ENA,"Kenai, AK",Alaska,United States,Alaska,30299,ANC,"Anchorage, AK",Alaska,United States,Alaska,4,416,1,2024,2,6,1,F,DU
314503,995.0,994.0,2737200.0,8946.0,7247.0,10718.0,0.0,59.0,37921.0,27463.0,GV,Grant Aviation,D,GV,Grant Aviation,5,30299,ANC,"Anchorage, AK",Alaska,United States,Alaska,31555,ENA,"Kenai, AK",Alaska,United States,Alaska,4,416,1,2024,3,7,1,F,DU
314504,999.0,998.0,2748200.0,8991.0,6631.0,5663.0,0.0,59.0,39218.0,30137.0,GV,Grant Aviation,D,GV,Grant Aviation,5,31555,ENA,"Kenai, AK",Alaska,United States,Alaska,30299,ANC,"Anchorage, AK",Alaska,United States,Alaska,4,416,1,2024,3,7,1,F,DU
314505,1522.0,1522.0,1826400.0,8026.0,5084.0,10340.0,0.0,11.0,17768.0,13202.0,S2,Star Marianas Air Inc.,D,S2,Star Marianas Air Inc.,6,35215,TIQ,"Tinian, TT",U.S. Pacific Trust Territories and Possessions,United States,U.S. Pacific Trust Territories and Possessions,34955,SPN,"Saipan, TT",U.S. Pacific Trust Territories and Possessions,United States,U.S. Pacific Trust Territories and Possessions,0,79,1,2024,3,7,1,F,DU


<h3>Filter Major Carriers</h3>

In [41]:
carrier_classes

,Code,Description
0,0,Foreign Carriers
1,1,Large Regional Carriers (carriers with annual ...
2,2,National Carriers (carriers with annual revenu...
3,3,Major Carriers (carriers with annual revenue o...
4,4,Medium Regional Carriers (carriers with annual...
5,5,Small Certificated Carriers (carrier holding c...
6,6,Commuter Carriers (air taxi operator which per...
7,7,All Cargo Carriers operating under cerificates...
8,9,Commuter Carriers (Air Taxi providing Essentia...


In [63]:
# Filter only major carriers (TranStats category 3)
major_carriers = big_table.loc[big_table['CARRIER_GROUP_NEW'] == 3]
major_carriers['CARRIER_NAME'].unique()

array(['Delta Air Lines Inc.', 'USA Jet Airlines Inc.',
       'United Air Lines Inc.', 'Republic Airline', 'JetBlue Airways',
       'Hawaiian Airlines Inc.', 'Allegiant Air', 'Alaska Airlines Inc.',
       'Southwest Airlines Co.', 'American Airlines Inc.',
       'Sun Country Airlines d/b/a MN Airlines', 'Atlas Air Inc.',
       'Federal Express Corporation', 'Polar Air Cargo Airways',
       'Kalitta Air LLC', 'Spirit Air Lines', 'SkyWest Airlines Inc.',
       'Envoy Air', 'Frontier Airlines Inc.', 'United Parcel Service'],
      dtype=object)

In [78]:
# Filter passenger flights only
major_passenger_flights = major_carriers.loc[major_carriers['PASSENGERS'] > 0]
major_passenger_flights['CARRIER_NAME'].unique()

array(['United Air Lines Inc.', 'Republic Airline', 'JetBlue Airways',
       'Delta Air Lines Inc.', 'Hawaiian Airlines Inc.', 'Allegiant Air',
       'Alaska Airlines Inc.', 'Southwest Airlines Co.',
       'American Airlines Inc.', 'Sun Country Airlines d/b/a MN Airlines',
       'Atlas Air Inc.', 'SkyWest Airlines Inc.', 'Envoy Air',
       'Spirit Air Lines', 'Frontier Airlines Inc.'], dtype=object)

In [90]:
# Filter flights performed
major_passenger_flights = major_passenger_flights[major_passenger_flights['DEPARTURES_PERFORMED'] > 10]

<h3>Load Factor Analysis</h3>

In [152]:
# Add load factor metric
major_passenger_flights['LF'] = major_passenger_flights['PASSENGERS']/major_passenger_flights['SEATS']

<h6>Best LF overall</h6>

In [144]:
# Select and reorder relevant columns
mpf_abbrv = major_passenger_flights.loc[:, ['ORIGIN', 'DEST', 'CARRIER_NAME', 'DEPARTURES_PERFORMED', 'SEATS', 'PASSENGERS', 'LF', 'DISTANCE']]
# Sort individual carrier routes by load factor
mpf_abbrv.sort_values(by='LF', ascending=False)

,ORIGIN,DEST,CARRIER_NAME,DEPARTURES_PERFORMED,SEATS,PASSENGERS,LF,DISTANCE
196194,BWI,IAH,Spirit Air Lines,12.0,2184.0,2194.0,1.004579,1235.0
192571,BDL,CLT,American Airlines Inc.,11.0,1408.0,1403.0,0.996449,644.0
203111,LAS,BUF,Southwest Airlines Co.,14.0,2002.0,1990.0,0.994006,1986.0
197289,PUJ,BWI,Southwest Airlines Co.,12.0,2100.0,2082.0,0.991429,1504.0
195573,FCA,ORD,United Air Lines Inc.,12.0,1512.0,1498.0,0.990741,1352.0
...,...,...,...,...,...,...,...,...
202524,SLC,CDC,SkyWest Airlines Inc.,14.0,700.0,81.0,0.115714,221.0
241527,ORD,MCW,SkyWest Airlines Inc.,27.0,1350.0,153.0,0.113333,288.0
205512,LAX,GJT,SkyWest Airlines Inc.,15.0,1020.0,114.0,0.111765,655.0
223572,MCW,ORD,SkyWest Airlines Inc.,21.0,1050.0,112.0,0.106667,288.0


<h6>Best LF domestic routes</h6>

In [143]:
# Filter domestic routes
mpf_dom_abbrv = major_passenger_flights[major_passenger_flights['REGION'] == 'D']
# Select and order specified columns
mpf_dom_abbrv = mpf_dom_abbrv.loc[:, ['ORIGIN', 'DEST', 'CARRIER_NAME', 'DEPARTURES_PERFORMED', 'SEATS', 'PASSENGERS', 'LF', 'DISTANCE']]
# Sort individual carrier routes by load factor
mpf_dom_abbrv.sort_values(by='LF', ascending=False)

,ORIGIN,DEST,CARRIER_NAME,DEPARTURES_PERFORMED,SEATS,PASSENGERS,LF,DISTANCE
196194,BWI,IAH,Spirit Air Lines,12.0,2184.0,2194.0,1.004579,1235.0
192571,BDL,CLT,American Airlines Inc.,11.0,1408.0,1403.0,0.996449,644.0
203111,LAS,BUF,Southwest Airlines Co.,14.0,2002.0,1990.0,0.994006,1986.0
195573,FCA,ORD,United Air Lines Inc.,12.0,1512.0,1498.0,0.990741,1352.0
193814,SLC,BWI,Southwest Airlines Co.,11.0,1925.0,1904.0,0.989091,1865.0
...,...,...,...,...,...,...,...,...
202524,SLC,CDC,SkyWest Airlines Inc.,14.0,700.0,81.0,0.115714,221.0
241527,ORD,MCW,SkyWest Airlines Inc.,27.0,1350.0,153.0,0.113333,288.0
205512,LAX,GJT,SkyWest Airlines Inc.,15.0,1020.0,114.0,0.111765,655.0
223572,MCW,ORD,SkyWest Airlines Inc.,21.0,1050.0,112.0,0.106667,288.0


<h6>Best LF international routes</h6>

In [142]:
# Filter international routes
mpf_int_abbrv = major_passenger_flights[major_passenger_flights['REGION'] != 'D']
# Select and order specified columns
mpf_int_abbrv = mpf_int_abbrv.loc[:, ['ORIGIN', 'DEST', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME','CARRIER_NAME', 'DEPARTURES_PERFORMED', 'SEATS', 'PASSENGERS', 'LF', 'DISTANCE']]
# Sort individual carrier routes by load factor
mpf_int_abbrv.sort_values(by='LF', ascending=False)

,ORIGIN,DEST,ORIGIN_CITY_NAME,DEST_CITY_NAME,CARRIER_NAME,DEPARTURES_PERFORMED,SEATS,PASSENGERS,LF,DISTANCE
197289,PUJ,BWI,"Punta Cana, Dominican Republic","Baltimore, MD",Southwest Airlines Co.,12.0,2100.0,2082.0,0.991429,1504.0
242839,SJD,ORD,"San Jose del Cabo, Mexico","Chicago, IL",American Airlines Inc.,27.0,4644.0,4601.0,0.990741,1807.0
261540,PUJ,DFW,"Punta Cana, Dominican Republic","Dallas/Fort Worth, TX",American Airlines Inc.,31.0,5332.0,5274.0,0.989122,2032.0
194163,PUJ,PHL,"Punta Cana, Dominican Republic","Philadelphia, PA",American Airlines Inc.,11.0,2090.0,2063.0,0.987081,1523.0
266694,SXM,CLT,"Philipsburg, Sint Maarten","Charlotte, NC",American Airlines Inc.,33.0,4223.0,4168.0,0.986976,1612.0
...,...,...,...,...,...,...,...,...,...,...
194011,SJU,PUJ,"San Juan, PR","Punta Cana, Dominican Republic",Frontier Airlines Inc.,11.0,2640.0,578.0,0.218939,155.0
249741,JFK,PAP,"New York, NY","Port-au-Prince, Haiti",JetBlue Airways,29.0,4698.0,974.0,0.207322,1522.0
254814,GUM,NGO,"Guam, TT","Nagoya, Japan",United Air Lines Inc.,30.0,4980.0,1009.0,0.202610,1554.0
254986,NGO,GUM,"Nagoya, Japan","Guam, TT",United Air Lines Inc.,30.0,4980.0,1006.0,0.202008,1554.0


<h6>LF by carrier, domestic and international</h6>

In [191]:
# Select SEATS, and PASSENGERS columns and group by airline and region
seats_by_carrier = major_passenger_flights.groupby(['CARRIER_NAME', 'REGION']).agg({'SEATS': 'sum', 'PASSENGERS': 'sum'}).reset_index()
seats_by_carrier

,CARRIER_NAME,REGION,SEATS,PASSENGERS
0,Alaska Airlines Inc.,D,20510896.0,16716500.0
1,Alaska Airlines Inc.,L,1342808.0,1138302.0
2,Allegiant Air,D,5729634.0,4922041.0
3,American Airlines Inc.,A,6185120.0,4986334.0
4,American Airlines Inc.,D,93467465.0,78873816.0
5,American Airlines Inc.,L,16716060.0,14264102.0
6,American Airlines Inc.,P,830777.0,696810.0
7,Delta Air Lines Inc.,A,7645850.0,6434524.0
8,Delta Air Lines Inc.,D,91611764.0,79191588.0
9,Delta Air Lines Inc.,L,7785702.0,6849894.0


In [192]:
# Pivot to create separate columns for each region and metric
pivoted_regions = seats_by_carrier.pivot_table(index='CARRIER_NAME', columns='REGION', values=['SEATS', 'PASSENGERS'], fill_value=0)

# Flatten the MultiIndex in columns and rename columns
pivoted_regions.columns = [f'{metric}_{region}' for metric, region in pivoted_regions.columns]

# Reset index to turn CARRIER_NAME back into a column
pivoted_regions = pivoted_regions.reset_index()

# Display the resulting table
pivoted_regions

,CARRIER_NAME,PASSENGERS_A,PASSENGERS_D,PASSENGERS_L,PASSENGERS_P,SEATS_A,SEATS_D,SEATS_L,SEATS_P
0,Alaska Airlines Inc.,0,16716500,1138302,0,0,20510896,1342808,0
1,Allegiant Air,0,4922041,0,0,0,5729634,0,0
2,American Airlines Inc.,4986334,78873816,14264102,696810,6185120,93467465,16716060,830777
3,Delta Air Lines Inc.,6434524,79191588,6849894,1389322,7645850,91611764,7785702,1762540
4,Envoy Air,0,9306590,657913,0,0,11296556,833969,0
5,Frontier Airlines Inc.,0,13239933,814283,0,0,17175174,1141420,0
6,Hawaiian Airlines Inc.,0,5805535,0,532060,0,7108154,0,700004
7,JetBlue Airways,451528,16608247,5814697,0,613314,20091800,7241905,0
8,Republic Airline,0,10327495,0,0,0,13001015,0,0
9,SkyWest Airlines Inc.,0,22666354,229514,0,0,28182274,283894,0


In [194]:
# Lamda function to categorize flights as domestic/international
major_passenger_flights['DOMESTIC'] = major_passenger_flights['REGION'].apply(lambda x: 'D' if x == 'D' else 'I')

# Group by CARRIER_NAME and DOMESTIC, then sum SEATS and PASSENGERS
seats_by_carrier = major_passenger_flights.groupby(['CARRIER_NAME', 'DOMESTIC']).agg({
    'SEATS': 'sum',
    'PASSENGERS': 'sum'
}).reset_index()

# Pivot to create separate columns for Domestic and International regions
pivoted_ID_seats_pax = seats_by_carrier.pivot_table(index='CARRIER_NAME', columns='DOMESTIC', values=['SEATS', 'PASSENGERS'], fill_value=0)

# Flatten the MultiIndex in columns and rename columns
pivoted_ID_seats_pax.columns = [f'{metric}_{region}' for metric, region in pivoted_ID_seats_pax.columns]

# Reset index to turn CARRIER_NAME back into a column
pivoted_ID_seats_pax = pivoted_ID_seats_pax.reset_index()

pivoted_ID_seats_pax

,CARRIER_NAME,PASSENGERS_D,PASSENGERS_I,SEATS_D,SEATS_I
0,Alaska Airlines Inc.,16716500,1138302,20510896,1342808
1,Allegiant Air,4922041,0,5729634,0
2,American Airlines Inc.,78873816,19947246,93467465,23731957
3,Delta Air Lines Inc.,79191588,14673740,91611764,17194092
4,Envoy Air,9306590,657913,11296556,833969
5,Frontier Airlines Inc.,13239933,814283,17175174,1141420
6,Hawaiian Airlines Inc.,5805535,532060,7108154,700004
7,JetBlue Airways,16608247,6266225,20091800,7855219
8,Republic Airline,10327495,0,13001015,0
9,SkyWest Airlines Inc.,22666354,229514,28182274,283894


In [213]:
LF_by_carrier = pivoted_ID_seats_pax
LF_by_carrier['Total LF'] = (pivoted_ID_seats_pax['PASSENGERS_D'] + pivoted_ID_seats_pax['PASSENGERS_I']) / (pivoted_ID_seats_pax['SEATS_D'] + pivoted_ID_seats_pax['SEATS_I'])
LF_by_carrier['Domestic LF'] = pivoted_ID_seats_pax['PASSENGERS_D'] / pivoted_ID_seats_pax['SEATS_D']
LF_by_carrier['International LF'] = pivoted_ID_seats_pax['PASSENGERS_I'] / pivoted_ID_seats_pax['SEATS_I']
LF_by_carrier = LF_by_carrier.loc[:, ['CARRIER_NAME', 'Total LF', 'Domestic LF', 'International LF']]
LF_by_carrier.sort_values(by='Total LF', ascending=False)

,CARRIER_NAME,Total LF,Domestic LF,International LF
3,Delta Air Lines Inc.,0.862686,0.864426,0.853418
1,Allegiant Air,0.859050,0.859050,NaN
12,Sun Country Airlines d/b/a MN Airlines,0.856746,0.865843,0.798086
2,American Airlines Inc.,0.843187,0.843864,0.840523
13,United Air Lines Inc.,0.840662,0.851609,0.807783
11,Spirit Air Lines,0.822254,0.828956,0.766056
4,Envoy Air,0.821440,0.823843,0.788894
7,JetBlue Airways,0.818494,0.826618,0.797715
0,Alaska Airlines Inc.,0.817015,0.815006,0.847703
6,Hawaiian Airlines Inc.,0.811663,0.816743,0.760081
